In [4]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import pickle as pk
import resources.RES as res
from sklearn.preprocessing import minmax_scale as sk_minmax


In [5]:
# retrieve context, maximum extents, and maximum extends as voxelised volume

context_path = os.path.relpath("../data/movedcontext.obj") 
context_mesh = tm.load(context_path)

# extents_path = os.path.relpath("../data/maximumextents.obj") 
# extents_mesh = tm.load(extents_path)

env_lat_path = os.path.relpath("../data/macrovoxels.csv")
envelope_lattice = tg.lattice_from_csv(env_lat_path)

In [6]:
# PV positions
unit = envelope_lattice.unit[0]
dist = unit/2 # distance from centroid to top of voxel

transfrm = [0,0,dist] 
ctr = envelope_lattice.centroids

newctr = ctr + transfrm  # move z-value of centroids for placement/calculation of PV

In [7]:
# ############## PV rotation NOT NEEDED CURRENTLY/OLD ####################
# azimuth_angle = 10 # degrees (counterclockwise) the panels are rotated from north
# azimuth = m.radians(azimuth_angle + 180) # northern hemisphere

# pv_angle = -15 # degrees of tilt for the pv panels on the roof (0 = horizontal, 90 = vertical)
# pv_tilt = m.radians(pv_angle)

# # calculate xyz components of surface normal vector (plane)
# Vx, Vy, Vz = m.sin(pv_tilt) * m.sin(azimuth), - m.sin(pv_tilt) * m.cos(azimuth), m.cos(pv_tilt)
# PV_normals = np.tile([Vx, Vy, Vz], (len(newctr),1)) # create vectors for each position

In [8]:
# # visualize normal vectors
# p = pv.Plotter(notebook=True)

# def tri_to_pv(tri_mesh):
#     faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
#     pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
#     return pv_mesh

# # fast visualization of the lattice
# envelope_lattice.fast_vis(p)

# # add the normal vectors
# p.add_arrows(newctr, PV_normals, mag=25)

# # add solar vector
# sunvectors = pk.load(open("../data/sunvectors.pk", "rb"))
# p.add_arrows(-300*sunvectors, sunvectors, mag = 25)

# # add context
# p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')

# # add extents
# # p.add_mesh(extents_mesh)

# # plotting
# p.show(use_ipyvtk=True)

In [9]:
# retrieve solar position, hour of the year, and irradiation values

In [10]:
# sunvectors
sunvectors = pk.load(open("../data/sunvectors.pk", "rb"))

In [11]:
# direct normal radiation
dnr = pk.load(open("../data/dnrval.pk", "rb"))

In [12]:
# hour of the year
hoy = pk.load(open("../data/hoys.pk", "rb"))

In [13]:
# # calculating performance over a time period in kWh ################# OLD ######################

# # using: https://photovoltaic-software.com/principle-ressources/how-calculate-solar-energy-power-pv-systems 
# # E = Energy (kWh)
# # A = Total solar panel Area (m2)
# # r = Solar panel yield or efficiency(%) - 15% or something
# # PR = Performance ratio, coefficient for losses (range between 0.5 and 0.9, default value = 0.75)
# # H = Average solar radiation on tilted panels (shadings not included) = Smodule over the time period (summed)

# #constants:
# A = 1
# r = 0.15
# PR = 0.75

# # calculate yield from surface normal vector (by defining rotation and tilt of panels) and Solar radiation perpendicular to sun and the solar vectors from GH
# # Smodule = SincidentS.N where Sincident is solar rad perp to sun, S is unit vector to sun, N is unit vector normal to surface
# # from: https://www.pveducation.org/pvcdrom/properties-of-sunlight/arbitrary-orientation-and-tilt 

# S = -sunvectors
# N = PV_normals
# S_incident = dnr

# # print(S.shape, N.shape, S_incident.shape)
# # print(S.T.shape, N.T.shape, S_incident.T.shape)
# #H = S_incident * np.dot(S,N)
# #Htotal = np.sum(H)
# S_module = np.dot(N, (S.T)) * S_incident
# cum_rad = np.sum(S_module,-1) # cumulative irradiation values over the year: these are currently uniform since every voxel and every sun vector is the same

# E = A * r * PR * cum_rad

# # TODO: replace blocked/unblocked vectors with 0's according to status e.g. if a ray is blocked by surrounding buildings, the ray becomes a 0 for that voxel
# # TODO: add interdependancy for for rays so mcda can be applied for selecting best interblocking voxels

In [14]:
# create trimesh cuboids for computing intersections
def transform_mat(value):
    mat = np.identity(4)
    mat[:3,-1] = np.array(value)
    return mat

In [15]:
# voxel cuboid meshes
vox_cuboids = [tm.creation.box(envelope_lattice.unit, transform=transform_mat(ct)) for ct in envelope_lattice.centroids]

# number of faces per voxel
f_numb = len(vox_cuboids[0].faces)

# combine voxels into one mesh
combined_voxels = tm.util.concatenate(vox_cuboids)

In [16]:
# combine voxels and context into one mesh
combined_meshes = tm.util.concatenate(combined_voxels, context_mesh)

# voxel centroids
vox_ctr = envelope_lattice.centroids

PV_ray_ctr = np.tile(-sunvectors, [len(newctr),1]) # PV ray for each centroid
PV_ctr_ray = np.tile(newctr, [1, len(sunvectors)]).reshape(-1, 3) # PV centroid for each ray

In [17]:
# intersection of rays from voxel centroids to sun objects with all voxel faces
face_id, ray_id = combined_meshes.ray.intersects_id(ray_origins=PV_ctr_ray, ray_directions=PV_ray_ctr, multiple_hits=True) # environment too complex? Takes about 70s

In [18]:
G1, U1 = res.construct_graph(sunvectors, face_id, ray_id, envelope_lattice, f_numb)

In [19]:
# ############# OLD ############################################################################
# # save voxel interdependencies array
# G1_l_indices = np.argwhere(G1>0)
# # G1_l_vals = G1[tuple(G1_l_indices.T)]
# # G1_l_flat = np.hstack([G1_l_indices, G1_l_vals.reshape(G1_l_vals.size, -1)]).astype(int)
# G1_l_flat = np.insert(G1_l_indices, len(G1_l_indices[0]), 1, axis=1)
# pk.dump(G1_l_flat, open("../data/G1_l_flat.pk", "wb"))

# # save obstracted rays array
# np.savetxt("../data/U1.csv", U1, delimiter=',')

# # save cuboid meshes as one mesh
# file_string = tm.exchange.obj.export_obj(combined_meshes, include_normals=True)
# with open("../data/cuboids.obj", "w") as file:
#     file.write(file_string)

In [20]:
# sum per row twice
sol_obstructed = (G1.sum(axis=1)).sum(axis=1)

# sum per column and then per row
sol_obstructing = (G1.sum(axis=0)).sum(axis=1)

In [21]:
def reshape_and_store_to_lattice(values_list, envelope_lattice):
    env_all_vox_id = envelope_lattice.indices.flatten()
    env_all_vox = envelope_lattice.flatten() # envelope inclusion condition: True-False
    env_in_vox_id = env_all_vox_id[env_all_vox] # keep in-envelope voxels (True)

    # initialize array
    values_array = np.full(env_all_vox.shape, 0.0)
    
    # store values for the in-envelope voxels
    values_array[env_in_vox_id] = values_list

    # reshape to lattice shape
    values_array_3d = values_array.reshape(envelope_lattice.shape)

    # convert to lattice
    values_lattice = tg.to_lattice(values_array_3d, envelope_lattice)

    return values_lattice

In [22]:
# apply weight to rays (direct normal radiation form hoy values)
w1 = dnr

# dot product with ray weights
o1 = np.array(np.dot(G1.sum(axis=1),w1), dtype='int64') # sun blocking
o2 = np.array(np.dot(G1.sum(axis=0),w1), dtype='int64') # sun blocked

# obscurity cost formula (hadamard product)
c1 = np.multiply(o1+1, o2+1)

# normalize values
c1_norm = sk_minmax(c1) # PV potential

# reshape and store to lattice
c1_lat = reshape_and_store_to_lattice(c1_norm, envelope_lattice)

In [23]:
# save normalized values for later use
pk.dump(c1_norm, open("../data/c1_norm.pk", "wb"))
# normalized values of sun blocking/sun blocked cost of each voxel

In [25]:
# visualize pv potential values 
p = pv.Plotter(notebook=True)

def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# fast visualization of the lattice
# envelope_lattice.fast_vis(p)

# add context
p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')

base_lattice = c1_lat
grid = pv.UniformGrid() # Create the spatial reference
grid.dimensions = np.array(base_lattice.shape) + 1 # Set the grid dimensions
grid.origin = base_lattice.minbound - 0.5 * base_lattice.unit # The bottom left corner of the data set
grid.spacing = base_lattice.unit # These are the cell sizes along each axis

# Add the data values to the cell data
# grid.point_arrays["Score"] = base_lattice.flatten(order="F")  # Flatten the Lattice
grid.cell_arrays["Score"] = base_lattice.flatten(order="F")  # Flatten the Lattice
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])*1.0
p.add_volume(grid, cmap="coolwarm", clim=[base_lattice.min(), base_lattice.max()],opacity=opacity, shade=True)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(788.6785661399646, 711.2621611399645, 723.5957336399646),
 (65.08283250000001, -12.333572500000002, 0.0),
 (0.0, 0.0, 1.0)]